[Home](Home.ipynb)

# Tractors in a Field

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/44271219881/in/photolist-2mhx3pZ-2c2DKXx-2as6zhv-NmqBzq-PqUqMH-LzHA8R-FNNuso-xBU6iK-uAsBMU-pW6m5Q-fK1gBZ-f8XJdV-f7Aafx-eetJjU-e3r8SW-cPN971-axMS5k-atxWV5-9QKseA-9KDXBq" title="The Farm"><img src="https://live.staticflickr.com/1897/44271219881_2cefe067c9.jpg" width="500" height="375" alt="The Farm"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Looking down from above, we see a tractor plowing a field.  The field is our ASCII art canvas in some stories.  However today we have Unicode. 

In other stories, our Field is an Argand Plane.  The tractor is on a track which puts it at every point in the field.  We model both Field and Tractor as Python classes.

In still other stories, we're literally simulating the agricultural activities of plowing and planting.  Perhaps we're using programs to remotely control our robot devices.

In practice, we have many reasons to consider this canonical pattern, of a raster beam painting a picture.  We may contrast the sequential visiting of cells to parallel processes that evaluate the cells more or less simultaneously.

In the Python world, we have numpy for performing "vectorized" operations on entire fields in one operation, no looping required. 

The tractor-based approach implemented here is more strictly sequential, with a controllable "visit order" in some types of Tractor.  That means the route taken through a field, visiting all the cells in turn, may not be fixed.

If you eyeball the source code for [tractor_1.py](tractor_1.py), you will see that it uses a dict as its base data structure.  

Often times, including in other Tractor implementations, we think of a rectangular matrix as more like a list of lists.  

However, given dictionary keys might serve as lookup coordinates, the dict-based version may be functionally the same, and end up saving memory.

In [2]:
! python tractor_1.py

<Tractor @ (0, 0); Fuel 29; Phrase JUST USE IT>

...........
...........
...........
...........
...........
...JUST USE
 IT........
...........
...........
...........
...........


The [farmworld.py](farmworld.py) code is rather similar.  One interesting aspect of the design pattern is we have access to the tractors in the field through the field. However we also have access to the field through the tractors.  

It's as if each tractor contains a memorized version of the field, which would be true of either a human or robot driven machine.

In [2]:
import farmworld

In [3]:
farmworld._test()

make a movie
Empty field, all is peaceful

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************

Showing the tractors in a list: 
Tractor(pos=[10, 10], facing=E, marker=O, fuel=100)
Tractor(pos=[10, 11], facing=W, marker=X, fuel=100)
===
A busy day begins...

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
**********OX********
********************
********************
********************
********************
********************
********************
****************

In [6]:
print(dir())

['In', 'Out', '_', '_5', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_exit_code', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_ih', '_ii', '_iii', '_oh', 'exit', 'farmworld', 'get_ipython', 'quit', 'tractor_1']


In [11]:
thefarm = farmworld.Farm(20,20)
thefarm   # fires __repr__

Farm(20,20) @ 4457466256

In [13]:
print(str(thefarm))  # str triggers __str__ which calls Farm.render(self)

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
********************



In [16]:
t1 = farmworld.Tractor(thefarm, pos=[10,10], marker="$", facing="N")
t1.plow()
print(str(thefarm))

********************
********************
********************
********************
********************
********************
********************
********************
********************
********************
**********$*********
********************
********************
********************
********************
********************
********************
********************
********************
********************



Let's look at the Tractor type initializer, the ```__init__```:

```python
    
    class Tractor:

        def __init__(self, farm , pos = [0,0], 
                     facing="N", marker="*" , fuel=100):
            self.thefarm = farm
            self.pos = pos
            self.facing = facing
            self.marker = marker
            self.thefarm.add(self)
            self.fuel = fuel
```

The default position is ```[0,0]```, in the middle of the farm.  However no marker (character) gets planted in the farm's field, until we actually plow the ground, by activating the ```plow``` method inside of ```Tractor```.  

We might separate ```plant``` and ```plow``` in a different design.  The ```plow``` method might be a synonym for the ```__next__``` method for example.

```python

    class Tractor:
        
    ...
    
        def plow(self, marker=None):
            if marker:
                self.marker = marker
            y,x = self.pos
            self.thefarm.field[y][x] = self.marker

```

Here's where the rubber meets the road, or the tractor meets the dirt, as through the list of lists ```self.thefarm.field``` is any tractor's way to address (plow in) said field.

Notice also that our Tractor is defined as a generator, meaning it implements the ```__next__``` method, such that by triggering ```__next__``` we cause the tractor to move in whatever direction it's facing, until reaching an edge, it which point it's not smart enough to do anything put sit there, as time passes.


```python

    class Tractor:
        
        ...
        
      
        def __next__(self):
            """
            Makes me an iterator
            """
            y,x = self.pos

            if self.fuel > 0:

                if self.facing   == "N":
                    if y > 0:
                        y -= 1
                    else:
                        raise StopIteration

                elif self.facing == "S":
                    if y < self.thefarm.h - 1:
                        y += 1
                    else:
                        raise StopIteration

                elif self.facing == "W":
                    if x > 0:
                        x -= 1
                    else:
                        raise StopIteration

                elif self.facing == "E":
                    if x < self.thefarm.w - 1:
                        x += 1
                    else:
                        raise StopIteration

                self.fuel -= 1
                self.pos = (y,x)

            else:  # outta gas
                raise StopIteration

            return self.thefarm.field[y][x]
  

```


By hooking all the tractors (we may have many) to the farm's tiktok method, we cause them all to advance.

```python

    class Farm:
        
        ...
        
        def ticktock(self):  # controller
            """tick tock o' the clock
            time marches on!
            Advance each tractor in the direction it's facing,
            ignoring stuck tractors already at a fence (some
            types of Tractor are smarter than others about fences).
            """
            for tractor in self.tractors:
                try:
                    next(tractor) # state changer
                except StopIteration:
                    pass  # harmless stuck tractor signal

            self.framenumber += 1
            return self.framenumber     
```    

In [17]:
next(t1)
next(t1)
next(t1)
next(t1)
t1.plow()
print(str(thefarm))

********************
********************
********************
********************
********************
********************
**********$*********
********************
********************
********************
**********$*********
********************
********************
********************
********************
********************
********************
********************
********************
********************



![](http://news.bbcimg.co.uk/media/images/49538000/jpg/_49538127_cropcircle_spl.jpg)

In [5]:
# %load tractor_2.py
"""
CropCircleTractor

Inherits from Tractor with same __next__ based raster pattern,
however in this subclass, planting a @ occurs when the underlying
complex number in the corresponding plane does not diverge after
10 iterations of z = z * z + c.  Creates a file of ASCII art best
viewed fixed width font, small font size.

EXAMPLE OUTPUT:  https://flic.kr/p/xyNXhN

(cl) MIT License 2015 by 4dsolutions.net
"""

from tractor_1 import Tractor, Field

class CropCircleTractor(Tractor):

    def config(self, x_scale, y_scale, x_offset, y_offset):
        self.x_scale, self.y_scale = x_scale, y_scale
        self.x_offset, self.y_offset = x_offset, y_offset

    def __next__(self):
        super().__next__()  # updates pos
        c = complex((self.col + self.y_offset) * self.y_scale, 
                    (self.row + self.x_offset) * self.x_scale)
        z = complex(0,0)
        # here is where we could add more iterations and also
        # start to add nuance, in terms of "shady characters"
        for _ in range(15):
            z = z*z + c
        if abs(z) <= 2:
            self.plant("🎃")
        elif abs(z) <= 100:
            self.plant("🐍")
        elif abs(z) <= 10000:
            self.plant("👀")
        return z
    
    def __iter__(self):
        return self

if __name__ == "__main__":
    the_field = Field(100, 250)
    the_field.add_tractor(CropCircleTractor)  # initialized as added
    the_tractor = the_field.Ts[0] # grab reference to instance
    the_tractor.marker = " "
    the_tractor.config(.025, .01, -50, -200)
    the_tractor.fuel_level = 100 * 250
    for z in the_tractor:
        if the_tractor.pos == (99, 249):
            break
    with open("mandelbrot.txt", "w") as fractal:
        print(the_field, file = fractal)



In [38]:
! python tractor_2.py

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/51391664278/in/dateposted-public/" title="mandelbrot_emoji"><img src="https://live.staticflickr.com/65535/51391664278_8b48e812ab_z.jpg" width="640" height="303" alt="mandelbrot_emoji"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

In [37]:
# %load mandelbrot.txt

Use thefarm.py to print successive frames, like of a movie, to investigate the path taken by your tractors, once you've set them in the field and initialized them.

The Farm or Field type keeps track of what tractors are on them, and may nudge each one to a next position, suggesting the passage of time.

For its part, the Tractor type has an internalized Field instance, much as a tractor driver would have an internalized mental model of the terrain, plus a plan of action.

In [33]:
# ! python thefarm.py

In [27]:
"*" * 20

'********************'

In [29]:
import emo_tractor


💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀👻👻👻👻👻💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀
💀💀💀💀💀💀💀💀💀💀

🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐅🐅🐅🐅🐅🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙
🐙🐙🐙🐙🐙🐙🐙🐙🐙🐙


In [1]:
"""
Field & Tractor

Tractor behaves as an iterator wrapping around a
field in a spiral by default, returning from lower
right (n-1, m-1) to upper left (0, 0) where n, m is
number of rows and columns respectively.

The TractorWriter takes a string and starts planting
it sequentially at a preset position.

(cl) MIT License 2015 by 4dsolutions.net 
"""


class Field(dict):
    """
    Field is a mapping, subclass of dict, with keys (x, y)
    """

    def __init__(self, rows, columns, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dims = (rows, columns)
        self.marker = "."
        self.Ts = [ ]  # add tractors to this list

    @property
    def rows(self):
        return self.dims[0]

    @property
    def columns(self):
        return self.dims[1]


    def __str__(self):
        """
        output the field as a string
        """
        s = ""
        for x in range(self.rows):
            s += "\n"
            for y in range(self.columns):
                s += self[x,y] if (x,y) in self else self.marker
        return s

    def __repr__(self):
        return "Field({}, {})".format(self.rows, self.columns)

    def add_tractor(self, T):
        """
        A tractor gains a reference to this very field when added thereto
        """
        the_gen = T(self)
        self.Ts.append( the_gen )


class Tractor:
    """
    An iterator, spirals through Field and rasters to top again by default
    """

    def __init__(self, my_field):
        self._myfield = my_field  # shows up when added to a field
        self.pos = (0, 0)  # changing internal state
        self.fuel_level = 150  # might run out of gas,

    def plant(self, the_crop):
        self._myfield[self.pos] = the_crop

    def __iter__(self):
        return self

    def __next__(self):
        """
        Spiralling algorithm.  If more columns to go, stay in
        current row. If end of column, start a next row, which
        may be top left if this was last row.  Decrement fuel 
        with each increment.
        """
        self.fuel_level -= 1  # decrement fuel
        if self.col + 1 < self._myfield.columns:
            self.pos = (self.row, self.col + 1)
        else:
            if self.row + 1 < self._myfield.rows:
                self.pos = (self.row + 1, 0)
            else:
                self.pos = (0, 0)
        return self.pos

    @property
    def row(self):
        return self.pos[0]

    @property
    def col(self):
        return self.pos[1]

    def __repr__(self):
        return "<Tractor @ {}; Fuel {}>".format(self.pos, self.fuel_level)

class TractorWriter(Tractor):

    def write(self, what, where):
        self.what = what
        self.start_point = where
        self.cnt = 0
        self.writing = False

    def __next__(self):
        """
        Plant what is to be written once the start position
        is reached.
        """
        if self.pos == self.start_point:
            self.writing = True
        if self.writing:
            if self.cnt == len(self.what):
                self.writing = False
                self.cnt = 0
            else:
                self.plant(self.what[self.cnt])
            self.cnt += 1
        super().__next__()  # updates pos
        
    def __repr__(self):
        return "<Tractor @ {}; Fuel {}; Phrase {}>".format(self.pos, self.fuel_level, self.what)

if __name__ == "__main__":
    the_field = Field(11, 11)
    the_field.add_tractor(TractorWriter)
    the_tractor = the_field.Ts[0]
    the_tractor.write("JUST USE IT", (5,3))
    for _ in range(121):
        next(the_tractor)
    print(the_tractor)
    print(the_field)


<Tractor @ (0, 0); Fuel 29; Phrase JUST USE IT>

...........
...........
...........
...........
...........
...JUST USE
 IT........
...........
...........
...........
...........
